# Session 1 — Visualizations

This notebook produces the figures for Session 1 (Introduction: What Is a Crisis?).

**Data sources:**
- Laeven, L. & Valencia, F. (2020). Systemic Banking Crises Database II. *IMF Economic Review*, 68(2), 307–361.
- Bank for International Settlements. Credit-to-GDP gaps. https://www.bis.org/statistics/c_gaps.htm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

## Figure 1: Timeline of Systemic Banking Crises

The crisis dates below come from Table A1 in Laeven & Valencia (2020),
which lists the start and end years of all 151 systemic banking crises
identified during 1970–2017.

We select a subset of well-known episodes for readability.

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# Crisis dates from Laeven & Valencia (2020), Table A1
# Source: IMF Working Paper WP/18/206, Appendix Tables
# Each tuple is (start_year, end_year)
# ──────────────────────────────────────────────────────────────────────

crises = {
    "Argentina":      [(1980, 1982), (1989, 1991), (1995, 1995), (2001, 2003)],
    "Brazil":         [(1990, 1994), (1994, 1998)],
    "Chile":          [(1976, 1976), (1981, 1985)],
    "Mexico":         [(1981, 1985), (1994, 1996)],
    "Thailand":       [(1983, 1983), (1997, 2000)],
    "Indonesia":      [(1997, 2001)],
    "South Korea":    [(1997, 1998)],
    "Japan":          [(1997, 2001)],
    "Turkey":         [(1982, 1984), (2000, 2001)],
    "Russia":         [(1998, 1998), (2008, 2009)],
    "Sweden":         [(1991, 1995), (2008, 2009)],
    "Finland":        [(1991, 1995)],
    "Norway":         [(1991, 1993)],
    "United Kingdom": [(2007, 2011)],
    "United States":  [(1988, 1988), (2007, 2011)],
    "Spain":          [(1977, 1981), (2008, 2012)],
    "Ireland":        [(2008, 2012)],
    "Greece":         [(2008, 2012)],
    "Iceland":        [(2008, 2012)],
    "Germany":        [(2008, 2009)],
    "France":         [(2008, 2009)],
    "Netherlands":    [(2008, 2009)],
    "Cyprus":         [(2011, 2015)],
}

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# Plot: Timeline of Systemic Banking Crises
# ──────────────────────────────────────────────────────────────────────

# --- Customisable parameters ---
YEAR_MIN = 1975
YEAR_MAX = 2018
BAR_HEIGHT = 0.6
FIG_WIDTH = 14
FIG_HEIGHT = 10
CRISIS_COLOR = "#c0392b"
GRID_COLOR = "#cccccc"
FONT_SIZE_LABELS = 11
FONT_SIZE_TITLE = 14
FONT_SIZE_TICKS = 10

# --- Sorting: group by region, then alphabetical ---
region_order = [
    # North America
    "United States",
    # Europe
    "Cyprus", "Finland", "France", "Germany", "Greece",
    "Iceland", "Ireland", "Netherlands", "Norway", "Spain",
    "Sweden", "United Kingdom",
    # Asia
    "Indonesia", "Japan", "South Korea", "Thailand",
    # Latin America
    "Argentina", "Brazil", "Chile", "Mexico",
    # Other
    "Russia", "Turkey",
]

countries = [c for c in region_order if c in crises]
n = len(countries)

fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))

for i, country in enumerate(countries):
    for start, end in crises[country]:
        duration = max(end - start, 0.5)  # minimum visible width
        ax.barh(i, duration, left=start, height=BAR_HEIGHT,
                color=CRISIS_COLOR, alpha=0.85, edgecolor="white", linewidth=0.5)

ax.set_yticks(range(n))
ax.set_yticklabels(countries, fontsize=FONT_SIZE_LABELS)
ax.set_xlim(YEAR_MIN, YEAR_MAX)
ax.set_xlabel("Year", fontsize=FONT_SIZE_LABELS)
ax.set_title("Systemic Banking Crises, Selected Countries (1975–2017)",
             fontsize=FONT_SIZE_TITLE, fontweight="bold", pad=15)

ax.xaxis.set_major_locator(plt.MultipleLocator(5))
ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
ax.tick_params(axis="x", labelsize=FONT_SIZE_TICKS)
ax.grid(axis="x", which="major", color=GRID_COLOR, linewidth=0.5, alpha=0.7)
ax.set_axisbelow(True)
ax.invert_yaxis()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

fig.text(0.14, 0.01,
         "Source: Laeven & Valencia (2020), Systemic Banking Crises Database II, IMF Economic Review.",
         fontsize=9, color="gray", style="italic")

plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.savefig("crisis_timeline.png", dpi=150, bbox_inches="tight")
plt.show()

## Figure 2: BIS Credit-to-GDP Gap — United States

The credit-to-GDP gap is defined as the difference between the credit-to-GDP ratio
and its long-run trend, estimated using a one-sided Hodrick-Prescott filter with
a smoothing parameter of λ = 400,000 (as recommended by the BIS for quarterly data).

We download the data directly from the BIS Statistical Warehouse.

**Data source:** Bank for International Settlements, [Credit-to-GDP gaps](https://www.bis.org/statistics/c_gaps.htm)

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# Download BIS credit-to-GDP gap data
# The BIS publishes this as a CSV via their statistics portal.
# URL: https://data.bis.org/topics/CREDIT_GAP/data
# We use the direct CSV download.
# ──────────────────────────────────────────────────────────────────────

BIS_URL = (
    "https://stats.bis.org/api/v1/data/BIS,WS_CREDIT_GAP,1.0/"
    "Q.US.P.A.C?format=csv"
)

try:
    bis_raw = pd.read_csv(BIS_URL)
    # The BIS CSV uses 'TIME_PERIOD' and 'OBS_VALUE'
    bis = bis_raw[["TIME_PERIOD", "OBS_VALUE"]].copy()
    bis.columns = ["date", "gap"]
    bis["date"] = pd.PeriodIndex(bis["date"], freq="Q").to_timestamp()
    bis["gap"] = pd.to_numeric(bis["gap"], errors="coerce")
    bis = bis.dropna().sort_values("date").reset_index(drop=True)
    data_loaded = True
    print(f"BIS data loaded: {len(bis)} observations, {bis['date'].min():%Y-Q1} to {bis['date'].max():%Y}")
except Exception as e:
    print(f"Could not download BIS data: {e}")
    print("Falling back to FRED data (QUSN628BIS).")
    data_loaded = False

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# Fallback: try FRED if BIS download failed
# FRED series QUSN628BIS = Credit-to-GDP gap for the US (from BIS)
# ──────────────────────────────────────────────────────────────────────

if not data_loaded:
    FRED_URL = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=QUSN628BIS"
    try:
        bis = pd.read_csv(FRED_URL, parse_dates=["observation_date"])
        bis.columns = ["date", "gap"]
        bis["gap"] = pd.to_numeric(bis["gap"], errors="coerce")
        bis = bis.dropna().sort_values("date").reset_index(drop=True)
        data_loaded = True
        print(f"FRED data loaded: {len(bis)} observations")
    except Exception as e2:
        print(f"FRED download also failed: {e2}")
        print("Please download the data manually from:")
        print("  https://fred.stlouisfed.org/series/QUSN628BIS")

In [ ]:
# ──────────────────────────────────────────────────────────────────────
# Plot: Credit-to-GDP Gap for the United States
# ──────────────────────────────────────────────────────────────────────

if data_loaded:

    # --- Customisable parameters ---
    FIG_WIDTH = 14
    FIG_HEIGHT = 5.5
    POS_COLOR = "#e74c3c"   # gap > 0 (overheating)
    NEG_COLOR = "#3498db"   # gap < 0 (deleveraging)
    THRESHOLD = 10           # BIS warning threshold (pp)
    THRESHOLD_COLOR = "#e74c3c"
    RECESSION_COLOR = "#d5d5d5"
    FONT_SIZE_LABELS = 11
    FONT_SIZE_TITLE = 14

    # NBER recession dates (peak to trough)
    recessions_us = [
        ("1973-11-01", "1975-03-01"),
        ("1980-01-01", "1980-07-01"),
        ("1981-07-01", "1982-11-01"),
        ("1990-07-01", "1991-03-01"),
        ("2001-03-01", "2001-11-01"),
        ("2007-12-01", "2009-06-01"),
        ("2020-02-01", "2020-04-01"),
    ]

    fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))

    # Shade recessions
    for start, end in recessions_us:
        ax.axvspan(pd.Timestamp(start), pd.Timestamp(end),
                   color=RECESSION_COLOR, alpha=0.6, zorder=0)

    # Fill positive/negative gap
    ax.fill_between(bis["date"], bis["gap"], 0,
                    where=bis["gap"] >= 0, color=POS_COLOR, alpha=0.4, label="Positive gap (credit boom)")
    ax.fill_between(bis["date"], bis["gap"], 0,
                    where=bis["gap"] < 0, color=NEG_COLOR, alpha=0.4, label="Negative gap (deleveraging)")

    # Gap line
    ax.plot(bis["date"], bis["gap"], color="black", linewidth=1.2)

    # BIS threshold
    ax.axhline(y=THRESHOLD, color=THRESHOLD_COLOR, linestyle="--",
               linewidth=1, alpha=0.7, label=f"BIS warning threshold ({THRESHOLD} pp)")

    # Zero line
    ax.axhline(y=0, color="black", linewidth=0.5)

    ax.set_xlabel("Year", fontsize=FONT_SIZE_LABELS)
    ax.set_ylabel("Credit-to-GDP gap (percentage points)", fontsize=FONT_SIZE_LABELS)
    ax.set_title("Credit-to-GDP Gap: United States",
                 fontsize=FONT_SIZE_TITLE, fontweight="bold", pad=15)
    ax.legend(loc="upper left", fontsize=9, framealpha=0.9)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # Add recession legend manually
    recession_patch = mpatches.Patch(color=RECESSION_COLOR, alpha=0.6, label="NBER recession")
    handles, labels = ax.get_legend_handles_labels()
    handles.append(recession_patch)
    ax.legend(handles=handles, loc="upper left", fontsize=9, framealpha=0.9)

    fig.text(0.14, -0.02,
             "Source: Bank for International Settlements, Credit-to-GDP gaps (https://www.bis.org/statistics/c_gaps.htm).\n"
             "Grey areas indicate NBER-dated recessions. The gap is the deviation of the credit-to-GDP ratio from\n"
             "its long-run trend (one-sided HP filter, λ = 400,000).",
             fontsize=9, color="gray", style="italic")

    plt.tight_layout(rect=[0, 0.06, 1, 1])
    plt.savefig("credit_gap_us.png", dpi=150, bbox_inches="tight")
    plt.show()
else:
    print("Data not available. See instructions above to download manually.")

---

## Customization Guide

### Figure 1 (Crisis Timeline)

You can customize the following:

- **Add or remove countries**: Edit the `crises` dictionary. Each entry is `"Country": [(start, end), ...]`.
- **Change the ordering**: Edit the `region_order` list.
- **Adjust the time window**: Change `YEAR_MIN` and `YEAR_MAX`.
- **Colors and sizing**: Modify `CRISIS_COLOR`, `FIG_WIDTH`, `FIG_HEIGHT`, `BAR_HEIGHT`, and font sizes.

### Figure 2 (Credit-to-GDP Gap)

- **Change the country**: Modify the BIS API URL. Replace `US` with the two-letter country code (e.g., `GB` for the UK, `JP` for Japan, `DE` for Germany).
- **Adjust the threshold line**: Change `THRESHOLD` (default is 10 pp, the BIS early warning level).
- **Update recession dates**: Edit `recessions_us` for the relevant country. NBER dates apply to the U.S. only.
- **Colors**: `POS_COLOR` (boom) and `NEG_COLOR` (deleveraging) control the fill colors.